In [5]:
!pip install gradio


  Using cached gradio-5.24.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.8.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached orjson-3.10.16-cp311-cp311-win_amd64.whl.metadata (42 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.11.4-py3-none-win_amd64.whl.metadata (26 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.46.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadat

In [7]:
# Install required packages (only run once)
!pip install tensorflow keras pillow matplotlib tqdm gradio

# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import string

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from keras.models import Model

import gradio as gr


In [11]:
def clean_captions(captions):
    table = str.maketrans('', '', string.punctuation)
    cleaned = {}
    for key, cap_list in captions.items():
        cleaned[key] = []
        for caption in cap_list:
            caption = caption.lower().translate(table)
            caption = ' '.join(caption.split())
            caption = 'startseq ' + caption + ' endseq'
            cleaned[key].append(caption)
    return cleaned


In [13]:
def extract_features_from_directory(directory):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    features = {}
    for img_name in tqdm(os.listdir(directory)):
        filename = os.path.join(directory, img_name)
        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = img_name.split('.')[0]
        features[image_id] = feature
    return features


In [15]:
def create_tokenizer(captions_dict):
    all_captions = []
    for key in captions_dict:
        all_captions.extend(captions_dict[key])
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(all_captions)
    return tokenizer

def max_length(captions_dict):
    return max(len(caption.split()) for cap_list in captions_dict.values() for caption in cap_list)


In [17]:
def define_model(vocab_size, max_length):
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


In [19]:
def generate_caption(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word.get(yhat)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text.replace('startseq', '').replace('endseq', '').strip()


In [34]:
# 📦 Install required packages
!pip install tensorflow gradio --quiet

# 📥 Import modules
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import gradio as gr

# 🧠 Load VGG16 base model (remove final layer)
base_model = VGG16()
vgg_model = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)

# ✨ Feature extractor with visual interpretation
def extract_visual_features(img_path):
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)

    features = vgg_model.predict(image).flatten()

    # Group features into symbolic parts
    edges = features[:1000]
    shapes = features[1000:2000]
    textures = features[2000:3000]
    patterns = features[3000:]

    # Compute averages for simplicity
    output = "🔍 **Visual Feature Summary**\n\n"
    output += f"🟦 Edges: {np.mean(edges):.4f}\n"
    output += f"🟩 Shapes: {np.mean(shapes):.4f}\n"
    output += f"🟫 Textures: {np.mean(textures):.4f}\n"
    output += f"🟪 Patterns: {np.mean(patterns):.4f}\n\n"

    output += "📘 These values reflect how strongly the model detected each visual category in your image.\n"
    output += "They're extracted from the penultimate layer of VGG16, often used for transfer learning."

    return output

# 🌐 Gradio interface
iface = gr.Interface(
    fn=extract_visual_features,
    inputs=gr.Image(type="filepath", label="📤 Upload an Image"),
    outputs="text",
    title="🧠 VGG16 Visual Feature Interpreter",
    description="Upload an image to get a summary of extracted visual features like Edges, Shapes, and Patterns.",
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
